In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter

%matplotlib qt

# import module with layers
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model_module")

from adaptive_layer import *

In [2]:
tf.random.set_seed(5)
np.random.seed(5)

In [3]:
def illustrate_train(name, y_label, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    epochs = range(1, len(static_f)+1)
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    plt.title(name, fontsize="xx-large")

    ax.set_ylabel(y_label, fontsize="large")
    ax.set_xlabel('Epochs', fontsize="large")

    ax.plot(epochs, static_f, label='ReLU Function')
    ax.plot(epochs, adaptive_f, label='Swish Function')
    ax.plot(epochs, adaptive_nn, label='Adaptive NN')
    ax.plot(epochs, adaptive_conv_nn, label='Adaptive Conv NN')

    ax.legend()
    
    plt.grid()
    plt.show()

In [4]:
def illustrate_test(name, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    
    x = ["ReLU Function", "Swish Function", "Adaptive NN", "Adaptive Conv NN"]
    y = [static_f, adaptive_f, adaptive_nn, adaptive_conv_nn]
    
    plt.title(name, fontsize="xx-large")
    ax.set_ylabel("Accuracy", fontsize="large")
    
    ax.bar(x, y)
    
    plt.grid()
    plt.show()

In [5]:
workbook = xlsxwriter.Workbook('bc_bigger_classical.xlsx')

In [5]:
def save_data(name, static_f_train, adaptive_f_train, adaptive_nn_train, adaptive_conv_nn_train, 
              static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test):
    
    epochs = len(static_f_train[0][:])
    worksheet = workbook.add_worksheet(name)
    
    worksheet.write(1, 1, name + " Training Loss")
    worksheet.write(1, 7, name + " Training Accuracy")
    worksheet.write(1, 13, name + " Test Accuracy")
    
    worksheet.write(2, 1, "Epochs")
    worksheet.write(2, 2, "ReLU Function")
    worksheet.write(2, 3, "Swish Function")
    worksheet.write(2, 4, "Adaptive NN")
    worksheet.write(2, 5, "Adaptive Conv NN")
    
    worksheet.write(2, 7, "Epochs")
    worksheet.write(2, 8, "ReLU Function")
    worksheet.write(2, 9, "Swish Function")
    worksheet.write(2, 10, "Adaptive NN")
    worksheet.write(2, 11, "Adaptive Conv NN")
    
    worksheet.write(2, 13, "ReLU Function")
    worksheet.write(2, 14, "Swish Function")
    worksheet.write(2, 15, "Adaptive NN")
    worksheet.write(2, 16, "Adaptive Conv NN")
    
    worksheet.write(3, 13, static_f_test)
    worksheet.write(3, 14, adaptive_f_test)
    worksheet.write(3, 15, adaptive_nn_test)
    worksheet.write(3, 16, adaptive_conv_nn_test)
    
    for i in range(0, epochs):
        worksheet.write(3+i, 1, i+1)
        worksheet.write(3+i, 2, static_f_train[0][i])
        worksheet.write(3+i, 3, adaptive_f_train[0][i])
        worksheet.write(3+i, 4, adaptive_nn_train[0][i])
        worksheet.write(3+i, 5, adaptive_conv_nn_train[0][i])
        
        worksheet.write(3+i, 7, i+1)
        worksheet.write(3+i, 8, static_f_train[1][i])
        worksheet.write(3+i, 9, adaptive_f_train[1][i])
        worksheet.write(3+i, 10, adaptive_nn_train[1][i])
        worksheet.write(3+i, 11, adaptive_conv_nn_train[1][i])
    
    workbook.close()

In [6]:
# dataset

data = pd.read_csv('../datasets/sonar_data.csv')
data = pd.DataFrame(data).to_numpy()
np.random.shuffle(data)

x = data[:, :-1]
y = data[:, -1]

x = np.array(x, dtype=np.float32)
y = np.array([out == 'M' for out in (y)], dtype=np.float32)

x_train, x_test = x[:160, :], x[160:, :]
y_train, y_test = y[:160], y[160:]

y_train = tf.constant(y_train, shape=(y_train.shape[0], 1))
y_test = tf.constant(y_test, shape=(y_test.shape[0], 1))

In [7]:
EPOCHS = 200
name = "Sonar Binary Classification"

# measurement repetitions
flag = measurements = 5

static_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_nn_hist = np.zeros(shape=[2, EPOCHS])
adaptive_conv_nn_hist = np.zeros(shape=[2, EPOCHS])

static_f_test = 0
adaptive_f_test = 0
adaptive_nn_test = 0
adaptive_conv_nn_test = 0

In [8]:
name = "Sonar Binary Classification"

In [9]:
# 1086 params
static_f_model = keras.Sequential([
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [10]:
# 1086 params
adaptive_f_model = keras.Sequential([
    keras.layers.Dense(15, activation=tf.nn.swish),
    keras.layers.Dense(10, activation=tf.nn.swish),
    keras.layers.Dense(1, activation="sigmoid")
])

In [11]:
# Adaptive Layer
# 353 params
adaptive_nn_model = AdaptiveModel([
    AdaptiveLayer(5, structure=[2, 1], inner_out_activation=tf.nn.sigmoid, skip_w=0.1),
    AdaptiveLayer(1, structure=[2, 1], inner_out_activation=tf.nn.sigmoid, skip_w=0.1)
])

In [12]:
# Adaptive Layer based on 1D Conv
# 997 params
adaptive_conv_nn_model = AdaptiveModel([
    AdaptiveLayerConv(5, structure=[2, 4], split=3, skip_w=0, noise=0, inner_out_activation=tf.nn.sigmoid),
    AdaptiveLayerConv(1, structure=[2, 4], skip_w=0, noise=0, inner_out_activation=tf.nn.sigmoid)
])

In [13]:
tmp = static_f_model(x_test)
tmp = adaptive_f_model(x_test)
tmp = adaptive_nn_model(x_test)
tmp = adaptive_conv_nn_model(x_test)

In [99]:
static_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.003), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [100]:
adaptive_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.003), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [101]:
adaptive_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.003), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [102]:
adaptive_conv_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.003), 
                       loss="binary_crossentropy", metrics=['accuracy'])

In [103]:
hist = static_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
3/3 [==============================] - 0s 1ms/step - loss: 0.7080 - accuracy: 0.4437
Epoch 2/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6975 - accuracy: 0.4750
Epoch 3/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6938 - accuracy: 0.5375
Epoch 4/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6921 - accuracy: 0.5312
Epoch 5/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6878 - accuracy: 0.5250
Epoch 6/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6839 - accuracy: 0.5250
Epoch 7/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6800 - accuracy: 0.5750
Epoch 8/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6750 - accuracy: 0.6625
Epoch 9/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6708 - accuracy: 0.6750
Epoch 10/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6650 - accuracy: 0.6875
Epoch 11/200
3/3 [===========

In [104]:
static_f_hist[0][:] += np.array(hist.history["loss"])/measurements
static_f_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [105]:
hist = adaptive_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6876 - accuracy: 0.5813
Epoch 2/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6811 - accuracy: 0.6000
Epoch 3/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6774 - accuracy: 0.5688
Epoch 4/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6752 - accuracy: 0.5562
Epoch 5/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6693 - accuracy: 0.5625
Epoch 6/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6631 - accuracy: 0.6062
Epoch 7/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6558 - accuracy: 0.6562
Epoch 8/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6479 - accuracy: 0.6625
Epoch 9/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6415 - accuracy: 0.6687
Epoch 10/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6315 - accuracy: 0.6812
Epoch 11/200
3/3 [===========

In [106]:
adaptive_f_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_f_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [107]:
hist = adaptive_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6932 - accuracy: 0.4688
Epoch 2/200
3/3 [==============================] - 0s 987us/step - loss: 0.6926 - accuracy: 0.5688
Epoch 3/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6924 - accuracy: 0.5125
Epoch 4/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6924 - accuracy: 0.5125
Epoch 5/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6920 - accuracy: 0.5125
Epoch 6/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6916 - accuracy: 0.5125
Epoch 7/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6912 - accuracy: 0.5125
Epoch 8/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6906 - accuracy: 0.5125
Epoch 9/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6901 - accuracy: 0.5125
Epoch 10/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6893 - accuracy: 0.5125
Epoch 11/200
3/3 [=========

In [108]:
adaptive_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_nn_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [109]:
hist = adaptive_conv_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6945 - accuracy: 0.4500
Epoch 2/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6925 - accuracy: 0.5125
Epoch 3/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6924 - accuracy: 0.5125
Epoch 4/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6932 - accuracy: 0.5125
Epoch 5/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6925 - accuracy: 0.5125
Epoch 6/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6920 - accuracy: 0.5125
Epoch 7/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6915 - accuracy: 0.5125
Epoch 8/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6905 - accuracy: 0.5125
Epoch 9/200
3/3 [==============================] - 0s 1ms/step - loss: 0.6904 - accuracy: 0.5437
Epoch 10/200
3/3 [==============================] - 0s 2ms/step - loss: 0.6894 - accuracy: 0.6812
Epoch 11/200
3/3 [===========

In [110]:
adaptive_conv_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_conv_nn_hist[1][:] += np.array(hist.history["accuracy"])/measurements

In [111]:
static_f_test += static_f_model.evaluate(x_test, y_test)[1]/measurements

2/2 [==============================] - 0s 2ms/step - loss: 0.4116 - accuracy: 0.8085


In [112]:
adaptive_f_test += adaptive_f_model.evaluate(x_test, y_test)[1]/measurements

2/2 [==============================] - 0s 2ms/step - loss: 0.4641 - accuracy: 0.8511


In [113]:
adaptive_nn_test += adaptive_nn_model.evaluate(x_test, y_test)[1]/measurements

2/2 [==============================] - 0s 2ms/step - loss: 0.3440 - accuracy: 0.9149


In [114]:
adaptive_conv_nn_test += adaptive_conv_nn_model.evaluate(x_test, y_test)[1]/measurements
flag -= 1
if(flag == 0):
    print("---------FINISH---------")

2/2 [==============================] - 0s 2ms/step - loss: 0.3384 - accuracy: 0.9149
---------FINISH---------


In [115]:
print("Static Function model\nTest accuracy : ", static_f_test)
print("\nAdaptive Function model\nTest accuracy : ", adaptive_f_test)
print("\nAdaptive NN model\nTest accuracy : ", adaptive_nn_test)
print("\nAdaptive Conv NN model\nTest accuracy : ", adaptive_conv_nn_test)

Static Function model
Test accuracy :  0.829787254333496

Adaptive Function model
Test accuracy :  0.8510638475418091

Adaptive NN model
Test accuracy :  0.9021276712417602

Adaptive Conv NN model
Test accuracy :  0.8978723526000977


In [116]:
illustrate_train(name+" Training Loss", "Loss", static_f_hist[0][:], 
                 adaptive_f_hist[0][:], adaptive_nn_hist[0][:], adaptive_conv_nn_hist[0][:])

In [117]:
illustrate_train(name + " Training Accuracy", "Accuracy", static_f_hist[1][:], 
                 adaptive_f_hist[1][:], adaptive_nn_hist[1][:], adaptive_conv_nn_hist[1][:])

In [118]:
illustrate_test(name + " Test Accuracy", 
                static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)

In [119]:
save_data(name, static_f_hist, adaptive_f_hist, adaptive_nn_hist, 
          adaptive_conv_nn_hist, static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)